In [1]:
!pip install -q langchain langchain-google-genai google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.7/233.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 29.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.46.0 which is incompatible.


In [2]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.46.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [10]:
!pip install -q pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 74.5 MB/s eta 0:00:00


In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from google.colab import files
from langchain_community.document_loaders import PyMuPDFLoader

In [8]:
GEMINI_API_KEY=userdata.get("GEMINI_API_KEY")
llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    api_key=GEMINI_API_KEY
)

In [44]:
uploaded=files.upload()

Saving story.pdf to story.pdf


In [45]:
file_name=list(uploaded.keys())[0]
loader=PyMuPDFLoader(file_name)
docs=loader.load()

In [46]:
combined_content=""
for doc in docs:
  combined_content+=doc.page_content

In [26]:
!pip install -q langchain-text-splitters

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [47]:
splitter=RecursiveCharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=0
)

In [48]:
chunks=splitter.split_text(combined_content)

In [49]:
combined_chunks = "\n".join(
    [f"Chunk {i+1}: {chunk}" for i, chunk in enumerate(chunks)]
)



In [50]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt=PromptTemplate(template="""
You are a professional English-to-Hindi translator.

Translate EACH English chunk below into Hindi.
Rules:
- Keep the chunk numbers exactly the same
- Do NOT merge chunks
- Do NOT skip any chunk
- Output format MUST be:

Chunk 1 (EN): <original text>
Chunk 1 (HI): <hindi translation>

Chunk 2 (EN): <original text>
Chunk 2 (HI): <hindi translation>

English Chunks:
{chunks}""",
input_variables=["chunks"])

chain= prompt | llm | StrOutputParser()

In [51]:
response=chain.invoke({
    "chunks":combined_chunks
})

In [52]:
print(response)

Chunk 1 (EN): Once upon a time in a small
Chunk 1 (HI): एक समय की बात है, एक छोटे से

Chunk 2 (EN): village nestled between the
Chunk 2 (HI): गाँव में, जो पहाड़ियों के बीच बसा हुआ था,

Chunk 3 (EN): hills, there lived a curious
Chunk 3 (HI): एक जिज्ञासु

Chunk 4 (EN): little girl named
Chunk 4 (HI): छोटी लड़की रहती थी जिसका नाम

Chunk 5 (EN): Meera. She loved asking
Chunk 5 (HI): मीरा था। उसे पूछना पसंद था

Chunk 6 (EN): questions and exploring the
Chunk 6 (HI): सवाल और अपने घर के पास

Chunk 7 (EN): woods near her home.
Chunk 7 (HI): जंगल को खोजना।

Chunk 8 (EN): Every morning, Meera would
Chunk 8 (HI): हर सुबह, मीरा

Chunk 9 (EN): set out on a new adventure,
Chunk 9 (HI): एक नए रोमांच पर निकल पड़ती थी,

Chunk 10 (EN): talking to birds and watching
Chunk 10 (HI): पक्षियों से बातें करती और

Chunk 11 (EN): squirrels
Chunk 11 (HI): गिलहरियों को

Chunk 12 (EN): dance on tree branches. Her
Chunk 12 (HI): पेड़ों की डालियों पर नाचते हुए देखती थी। उसकी

Chunk 13 (EN): favorite story was about 